In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import keras_nlp

2025-03-13 08:02:23.914715: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-13 08:02:24.123262: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-13 08:02:24.313635: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741852944.478112    4112 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741852944.522826    4112 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-13 08:02:24.925507: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
import pandas as pd
df=pd.read_csv('spam.csv')
df.head()


,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [4]:
df.groupby('Category').value_counts()

Category  Message                                                                                                                                                                     
ham       Sorry, I'll call later                                                                                                                                                          30
          I cant pick the phone right now. Pls send a message                                                                                                                             12
          Ok...                                                                                                                                                                           10
          7 wonders in My WORLD 7th You 6th Ur style 5th Ur smile 4th Ur Personality 3rd Ur Nature 2nd Ur SMS and 1st "Ur Lovely Friendship"... good morning dear                          4
          Ok                                                 

In [5]:
df['Category'].value_counts()

Category
ham     4825
spam     747
Name: count, dtype: int64

In [6]:
 747/4825

0.15481865284974095

In [7]:
df_spam=df[df['Category']=='spam']
df_spam.shape

(747, 2)

In [8]:
df_ham=df[df['Category']=='ham']
df_ham.shape

(4825, 2)

In [9]:
df_ham_downsampled=df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(747, 2)

In [10]:
df_balanced=pd.concat([df_spam,df_ham_downsampled])

In [11]:
df_balanced.shape

(1494, 2)

In [12]:
df_balanced['Category'].value_counts()

Category
spam    747
ham     747
Name: count, dtype: int64

In [13]:
df_balanced.sample(5)

,Category,Message
1094,ham,Well the weather in cali's great. But its comp...
1241,ham,Oh yes I can speak txt 2 u no! Hmm. Did u get ...
2583,spam,3 FREE TAROT TEXTS! Find out about your love l...
2975,ham,"I'll text carlos and let you know, hang on"
2328,spam,URGENT! Your mobile number *************** WON...


In [14]:
df_balanced['spam']=df_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0)

In [15]:
df_balanced.sample(5)

,Category,Message,spam
3064,spam,"Hi babe its Jordan, how r u? Im home from abro...",1
1312,ham,U r too much close to my heart. If u go away i...,0
3222,spam,"Thanks for your ringtone order, reference numb...",1
2881,spam,Last Chance! Claim ur £150 worth of discount v...,1
2989,ham,Do you still have the grinder?,0


In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df_balanced['Message'],df_balanced['spam'],stratify=df_balanced['spam'])

In [17]:
X_train.head(4)

367     Update_Now - Xmas Offer! Latest Motorola, Sony...
1916    Its not that time of the month nor mid of the ...
4499    Latest Nokia Mobile or iPOD MP3 Player +£400 p...
70         Wah lucky man... Then can save money... Hee...
Name: Message, dtype: object

In [18]:
bert_preprocess=hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder=hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

2025-03-13 08:02:36.430628: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-03-13 08:02:52.792889: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.


In [19]:
def get_sentence_embeding(sentences):
    preprocessed_text=bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [20]:
get_sentence_embeding([
    "Sunshine Hols. To claim ur med holiday",
    
                       ])

<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[-6.57662392e-01, -2.81492740e-01, -6.64784193e-01,
         5.82889438e-01,  3.52658033e-01,  3.01090330e-02,
         6.81208074e-01,  1.75162628e-01, -3.67962718e-01,
        -9.99834955e-01, -6.40497878e-02,  6.47660851e-01,
         9.50325191e-01,  3.24384868e-01,  7.35545576e-01,
        -4.17239606e-01,  9.94451195e-02, -4.39465374e-01,
         3.39418739e-01,  1.79578364e-02,  4.20951754e-01,
         9.98797774e-01,  3.34184945e-01,  9.56264883e-02,
         3.14250648e-01,  8.54856193e-01, -4.87252474e-01,
         7.92728126e-01,  8.96579623e-01,  5.84938109e-01,
        -3.84550989e-01,  1.43849343e-01, -9.66158390e-01,
        -1.40088931e-01, -7.45082200e-01, -9.64695990e-01,
         2.26245403e-01, -4.40775692e-01,  1.20793074e-01,
         1.34351239e-01, -6.34572983e-01,  2.11020827e-01,
         9.99314487e-01,  2.26245612e-01,  8.93676132e-02,
        -1.43336520e-01, -9.99903798e-01,  1.79020286e-01,
      

In [21]:
e=get_sentence_embeding([
    "banana",
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
])

In [22]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[1]])

array([[0.9911088]], dtype=float32)

In [23]:
cosine_similarity([e[0]],[e[4]])

array([[0.893363]], dtype=float32)

In [24]:
cosine_similarity([e[4]],[e[4]])

array([[0.9999998]], dtype=float32)

In [25]:
# Bert layers
# text_input=tf.keras.layers.Input(shape=(),dtype=tf.string,name="text")
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_base_en_uncased",trainable=True)
encoder_inputs = preprocessor(text_input)
encoder = keras_nlp.models.BertBackbone.from_preset("bert_base_en_uncased")
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].
#Neural network layers
l=tf.keras.layers.Dropout(0.1,name='dropout')(outputs['pooled_output'])
l=tf.keras.layers.Dense(1,activation='sigmoid',name='output')(l)

# #construct final model
model=tf.keras.Model(inputs=[text_input],outputs=[l])





2025-03-13 08:03:02.227041: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
2025-03-13 08:03:02.413462: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
2025-03-13 08:03:02.444293: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
2025-03-13 08:03:06.659101: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.


In [26]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text (InputLayer)   │ (None)            │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert_text_classifi… │ [(None, 512),     │          0 │ text[0][0]        │
│ (BertTextClassifie… │ (None, 512),      │            │                   │
│                     │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert_backbone       │ [(None, 768),     │ 109,482,2… │ bert_text_classi… │
│ (BertBackbone)      │ (None, 512, 768)] │            │ bert_text_classi… │
│                     │                   │            │ bert_text_classi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 768)       │          0 │ bert_backbone[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │        769 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 109,483,009 (417.64 MB)

 Trainable params: 109,483,009 (417.64 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
METRICS=[
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=METRICS)

: 

In [ ]:
model.fit(X_train,y_train,epochs=10)

Epoch 1/10


/workspaces/keras-exercise/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['text']. Received: the structure of inputs=*
  warnings.warn(


In [ ]:
y_predicted=model.preduct(X_test)
y_predicted=y_predicted.flatten()

In [ ]:
import numpy as np
y_predicted=np.where(y_predicted>0.5,1,0)
y_predicted

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
cm=confusion_matrix(y_test,y_predicted)
cm

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sn
sn.heatmap(cm,annot=True,fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test,y_predicted))